In [1]:
import girderformindlogger
import girder_client as gc
import pandas as pd
import numpy as np

In [2]:
girder = gc.GirderClient(host="localhost", port=8080)

# Plan

- [x] create a new user (userA)
- [x] get userA's applets and make sure the list is empty
- [x] userA creates an applet with an activity-set-url (url TBD)
- [x] get userA's applets and make sure we have the applet they just created
- [x] create userB
- [x] userA invites userB to the applet
- [x] make sure userB has an invite
- [x] userB accepts their invite
- [x] list userB's applets
- [x] userA invites userC (by email) to the applet
- [ ] check that the applet user table has userC pending
- [x] create userC with the same email
- [ ] check that userC has an invite
- [ ] accept userC's applet invite
- [ ] check userC's applets
- [ ] userA, B and C submit responses to each activity in the applet
- [ ] userA can see user's B & C's data
- [ ] get responses for each user in the last 7 days
- [ ] user B removes the applet without removing data
- [ ] user C removes the applet AND removes data
- [x] delete userB and userC
- [ ] userA deactivites their applet
- [ ] delete userA


# Create a new user

In [3]:
def testCreateUser(email=None):
    randomUser = np.random.randint(1000000)
    firstName = 'test'
    lastName = 'user'
    # NOTE: girder makes login and email lowercase!!!
    login = 'testuser{}'.format(randomUser)
    if not email:
        email = 'testemail{}@testemail.com'.format(randomUser)
    password = 'password'
    createUser = girder.post('user', parameters=dict(firstName=firstName,
                                        lastName=lastName,
                                        login=login,
                                        email=email,
                                        password=password
                                       )
               )
    # assert 'authToken' in createUser.keys(), 'user has no token, {}'.format(createUser)
    assert createUser['email'] == email, 'email does not match, {} {}'.format(createUser['email'], email)
    assert createUser['firstName'] == firstName, 'firstName does not match'
    assert createUser['lastName'] == lastName, 'lastName does not match'
    assert createUser['login'] == login, 'login does not match, {} {}'.format(createUser['login'], login)
    assert createUser['admin'] == False, 'user is an admin'
    assert createUser['_modelType'] == 'user', 'model is not user'
    assert createUser['public'] == False, 'user is public!'

    return createUser

In [4]:
def authenticate(user):
    girder.authenticate(username=user['login'], password='password')

In [5]:
userA = testCreateUser()
userB = testCreateUser()

#authenticate(userA)

# Get applets when the user doesn't have any

In [6]:
def getAppletsNewUser(new_user):
    girder.authenticate(username=new_user['login'], password='password')
    appletList = girder.get('user/applets')
    assert len(appletList) == 0, 'a new user should have an empty list of applets. we get {}'.format(appletList)
    return 1

In [7]:
getAppletsNewUser(userA)

1

# get Applet by ID

want to check recently added applet here

In [8]:
def getAppletById(user, ar):
    girder.authenticate(username=user['login'], password='password')
    res = girder.get('applet/{}'.format(ar['_id']))
    assert res['applet']['_id'].split('applet/')[1] == ar['_id'], 'applet ids are not the same'
    return 1

# Add an applet

In [9]:
def addApplet(new_user):
    girder.authenticate(username=new_user['login'], password='password')

    # TODO: create an activity-set that JUST for testing.
    # make sure it has all the weird qualities that can break


    # for now, lets do the mindlogger demo
    activitySetUrl = 'http://repronim.org/schema-standardization/activity-sets/mindlogger-demo/mindlogger-demo_schema'
    randomAS = np.random.randint(1000000)
    ar = girder.post('applet', parameters=dict(activitySetUrl = activitySetUrl, name='testActivitySet{}'.format(randomAS)))

    assert ar['_id'], 'there is no ID!'
    assert ar['meta']['activitySet']['url'] == activitySetUrl, 'the URLS do not match! {} {}'.format(ar['meta']['activitySet']['url'], activitySetUrl)
    
    assert getAppletById(new_user, ar) == 1, 'something wrong with getAppletById'
    return ar

In [10]:
userA_applets = addApplet(userA)

In [11]:
def getAppletsUser(user, n=1):
    girder.authenticate(username=user['login'], password='password')
    appletList = girder.get('user/applets')
    assert len(appletList) == n, 'a new user should have {} applets. we get {}'.format(n, len(appletList))
    return 1

In [12]:
getAppletsUser(userA, 1)

1

# Invite userB to userA's applet

In [29]:
def inviteUserToApplet(userA, userA_applets, userB):
    groupId = userA_applets['roles']['user']['groups'][0]['id']
    authenticate(userA)
    inviteResp = girder.post('group/{}/invitation'.format(groupId), {
        "email": userB['email']
    })
    assert inviteResp['public'] == False, 'invite is public!'
    assert len(inviteResp['access']['users']), 'ths user was not added'
    return inviteResp

In [14]:
inviteB = inviteUserToApplet(userA, userA_applets, userB)

## Get a group's pending invites and check for a user

In [15]:
def checkInvitesForUser(userA, userA_applets, userB):
    authenticate(userA)
    groupId = userA_applets['roles']['user']['groups'][0]['id']
    pendingInvites = girder.get('group/{}/invitation'.format(groupId))
    value = False
    for invite in pendingInvites:
        if invite['email'] == userB['email']:
            value = True
            break
    assert value, "email not in list of pending invites, {}".format(pendingInvites)
    return 1

In [16]:
checkInvitesForUser(userA, userA_applets, userB)

1

### Accept an applet invite

In [17]:
def checkForInvite(userB, userA_applets):
    authenticate(userB)
    pendingInvitesForUser = girder.get('user/invites')
    groupId = userA_applets['roles']['user']['groups'][0]['id']
    assert len(pendingInvitesForUser), "this user has no invites"
    assert pendingInvitesForUser[0]['_id'] == groupId, "this user doesn't have the invite you expect"
    return groupId

In [18]:
def acceptAppletInvite(userB, userA_applets):
    groupId = checkForInvite(userB, userA_applets)
    authenticate(userB)
    resp = girder.post('group/{}/member'.format(groupId))
    assert resp['_modelType'] == 'group', "something weird about response, {}".format(resp)
    return 1

In [19]:
acceptAppletInvite(userB, userA_applets)

1

get a list of applet users (manager)

In [48]:
def getUserTable(userA, userA_applets):
    authenticate(userA)
    appletUsers = girder.get('applet/{}/users'.format(userA_applets['_id']))
    return appletUsers

check that userB has an applet

In [20]:
getAppletsUser(userB, 1)

1

### Invite someone who does not have an account yet

In [56]:
userCemail = 'randomuserc{}@test.com'.format(np.random.randint(1000000))
inviteC = inviteUserToApplet(userA, userA_applets, {'email': userCemail})

In [57]:
inviteC

{'_accessLevel': 2,
 '_id': '5d8506c631c0900f975f43ea',
 '_modelType': 'group',
 'access': {'users': [{'flags': [],
    'id': '5d8506c531c0900f975f43e2',
    'level': 2}]},
 'created': '2019-09-20T17:05:10.123000+00:00',
 'description': '',
 'name': 'Default testActivitySet210534 (5d8506c631c0900f975f43e9) Users',
 'openRegistration': False,
 'public': False,
 'requests': [],
 'updated': '2019-09-20T17:36:41.937678+00:00'}

In [71]:
def checkAppletUserTableForUser(userA, userA_applets, userCemail):
    ut = getUserTable(userA, userA_applets)
    assert len(list(filter(lambda x: x['email'] == userCemail, ut))) == 1
    return ut

In [72]:
checkAppletUserTableForUser(userA, userA_applets, userCemail)

[{'_id': '5d850e43b855eabc011391e9',
  'email': 'randomuserc493026@test.com',
  'groups': [{'_id': '5d8506c631c0900f975f43ea',
    'name': 'Default testActivitySet210534 (5d8506c631c0900f975f43e9) Users',
    'role': 'user',
    'status': 'pending'}]},
 {'_id': '5d8506c531c0900f975f43e2',
  'email': 'testemail550840@testemail.com',
  'groups': [{'_id': '5d8506c631c0900f975f43ea',
    'name': 'Default testActivitySet210534 (5d8506c631c0900f975f43e9) Users',
    'role': 'user',
    'status': 'active'}]},
 {'_id': '5d8506c531c0900f975f43e5',
  'email': 'testemail108472@testemail.com',
  'groups': [{'_id': '5d8506c631c0900f975f43ea',
    'name': 'Default testActivitySet210534 (5d8506c631c0900f975f43e9) Users',
    'role': 'user',
    'status': 'active'}]},
 {'_id': '5d8506c531c0900f975f43e2',
  'email': 'testemail550840@testemail.com',
  'groups': [{'_id': '5d8506c631c0900f975f43eb',
    'name': 'Default testActivitySet210534 (5d8506c631c0900f975f43e9) Editors',
    'role': 'editor',
    '

create that person's account

In [62]:
userC = testCreateUser(userCemail)

and check if they have an invite

In [63]:
checkInvitesForUser(userA, userA_applets, userC)

1

In [64]:
checkForInvite(userC, userA_applets)

'5d8506c631c0900f975f43ea'

# Delete a user

In [ ]:
def testDeleteUser(new_user):

    girder.authenticate(username=new_user['login'], password='password')

    deleteUser = girder.delete('user/{}'.format(new_user['_id']))

    assert deleteUser['message'] == 'Deleted user {}.'.format(new_user['login']), "{} does not equal {}".format(deleteUser['message'], 'Deleted user {}'.format(new_user['login']))
    
    return 1

In [ ]:
testDeleteUser(userA)
testDeleteUser(userB)
testDeleteUser(userC)